# Star formation history plots

This version matches galaxies with both subhalos and halos.  
However at this stage, there are no properties stored in either subhalos or halos apart from mass (I think).

In [ ]:
# Imports
import astropy.constants as c
import astropy.units as u
import h5py
h5py.enable_ipython_completer()
import numpy as np
import yaml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

In [ ]:
# Script parameters
file_parameters='SFH.yml'
displayParameters=True

# Read parameters from yaml file
parameters=yaml.load(open(file_parameters),Loader=yaml.FullLoader)
if displayParameters:
    for item in parameters:
        print("{:20s}: {}".format(item,parameters[item]))

haloFile=parameters['inputFiles']['haloFile']
subhaloFile=parameters['inputFiles']['subhaloFile']
galaxyFile=parameters['inputFiles']['galaxyFile']
sfhFile=parameters['inputFiles']['sfhFile']
save_prefix=parameters['save_prefix']
graphMin=parameters['graphs']['graphMin']
graphMax=parameters['graphs']['graphMax']
snapMin=parameters['snapshots']['snapMin']
snapMax=parameters['snapshots']['snapMax']
assert snapMin==snapMax # Currently not set up to do any different
baryonFraction=parameters['cosmo']['baryonFraction']

In [ ]:
# Read in halo table
with h5py.File(haloFile,'r') as f:
    # Get the data from the snapshot
    halos=f['Halos'][:]
    print('halos.dtype =',halos.dtype)
    
# Read in subhalo table
with h5py.File(subhaloFile,'r') as f:
    # Get the data from the snapshot
    subs=f['Halos'][:]
    print('subs.dtype =',subs.dtype)

# Read in galaxy table
with h5py.File(galaxyFile,'r') as f:    
    # Get the data from the snapshot
    gals=f['Galaxies'][:]
    print('gals.dtype =',gals.dtype)
    # Get the starting locations of each graph in the galaxy table
    graph_start=f['Graph_start_locations'][:]

# Read in SFH bins
with h5py.File(sfhFile,'r') as f:
    t=f['t'][:]
    dt=f['dt'][:]
    i_bin=f['i_bin'][:]
t[-1,:]

In [ ]:
# Create timebins for our selection
n_bin=i_bin[snapMin]
# Extract values just for this snapshot
t=t[snapMin,:n_bin]
dt=dt[snapMin,:n_bin]
# Reverse times to be lookback times at centre of bins
t_end=t[-1]
t_lookback=t_end-t+0.5*dt

In [ ]:
# Filter out the galaxies in our selection.
# Might be faster to include np.nonzero() on rhs, but this packages indices in a tuple
gals=gals[(graphMin<=gals['graph_ID']) & (gals['graph_ID']<=graphMax) & 
            (snapMin<=gals['snap_ID']) & (gals['snap_ID']<=snapMax) & gals['b_exists']==True]
n_gal=len(gals)

In [ ]:
# Match up galaxies with subhalos and/or halos
# Note that orphan galaxies can have no halo if there is no descendant for the host halo in the previous snap.
# Note that orphan galaxies have no subhalo.
b_first_warning=True
halo_loc=np.zeros(n_gal,dtype=np.int32)
sub_loc=np.zeros(n_gal,dtype=np.int32)
mask_halo_exists=np.full(n_gal,True,dtype=bool)
mask_sub_exists=np.full(n_gal,True,dtype=bool)
for i_gal in range(n_gal):
    matches=np.where(np.logical_and(halos['graph_ID']==gals['graph_ID'][i_gal],halos['halo_gid']==gals['halo_gid'][i_gal]))[0]
    if len(matches)!=1:
        mask_halo_exists[i_gal]=False
    else:
        halo_loc[i_gal]=matches[0]
    matches=np.where(np.logical_and(subs['graph_ID']==gals['graph_ID'][i_gal],subs['sub_gid']==gals['sub_gid'][i_gal]))[0]   
    if len(matches)!=1:
        mask_sub_exists[i_gal]=False
    else:
        sub_loc[i_gal]=matches[0]
halos=halos[halo_loc]
subs=subs[sub_loc]

In [ ]:
for plot in parameters['plots'].items():
    print(plot)
    if plot[1]:
        plotName=plot[0]
        plotProps=parameters[plotName]
        print(plotProps)
        plt.figure(figsize=plotProps['figsize'])
        plt.xscale(plotProps['xscale'])
        plt.yscale(plotProps['yscale'])
        plt.title(plotName)
        plt.xlabel(plotProps['xlabel'])
        plt.ylabel(plotProps['ylabel'])
        if plotName=='test':
            mass_stars=gals['mass_stars_disc']
            mask=mass_stars>1
            xdata=mass_stars[mask]
            ydata=np.sum(gals['mass_stars_disc_sfh'],1)[mask]
            plt.plot([np.min(xdata),np.max(xdata)],[np.min(xdata),np.max(xdata)],'k-')
        elif plotName=='sfh_time':
            mass_stars=gals['mass_stars_disc']+gals['mass_stars_bulge']
            mask=mass_stars>1
            xdata=t_lookback
            ydata1=np.sum(gals['mass_stars_bulge_sfh'][mask,:n_bin],0)/dt
            plt.plot(xdata,ydata1,'*',label='bulge')
            ydata2=np.sum(gals['mass_stars_disc_sfh'][mask,:n_bin],0)/dt
            plt.plot(xdata,ydata2,'*',label='disc')
            ydata=ydata1+ydata2
        elif plotName=='sfh_bulge_mass':
            xdata=t_lookback
            mass_stars=gals['mass_stars_disc']+gals['mass_stars_bulge']
            mask=np.log10(mass_stars)<9.
            ydata=np.mean(gals['mass_stars_bulge_sfh'][mask,:n_bin],0)/dt
            plt.plot(xdata,ydata,'-',label='       $M_*/$M$_\odot<10^9$')
            for log10_mass_min in range(9,11):
                mask=np.logical_and(log10_mass_min<=np.log10(mass_stars),np.log10(mass_stars)<log10_mass_min+1)
                ydata=np.mean(gals['mass_stars_bulge_sfh'][mask,:n_bin],0)/dt
                plt.plot(xdata,ydata,'-',label='$10^{'+str(log10_mass_min)+'}\leq M_*/$M$_\odot<10^{'+str(log10_mass_min+1)+'}$')
            mask=np.log10(mass_stars)>=11.
            ydata=np.mean(gals['mass_stars_bulge_sfh'][mask,:n_bin],0)/dt
        elif plotName=='sfh_disc_mass':
            xdata=t_lookback
            mass_stars=gals['mass_stars_disc']+gals['mass_stars_bulge']
            mask=np.log10(mass_stars)<9.
            ydata=np.mean(gals['mass_stars_disc_sfh'][mask,:n_bin],0)/dt
            plt.plot(xdata,ydata,'-',label='       $M_*/$M$_\odot<10^9$')
            for log10_mass_min in range(9,11):
                mask=np.logical_and(log10_mass_min<=np.log10(mass_stars),np.log10(mass_stars)<log10_mass_min+1)
                ydata=np.mean(gals['mass_stars_disc_sfh'][mask,:n_bin],0)/dt
                plt.plot(xdata,ydata,'-',label='$10^{'+str(log10_mass_min)+'}\leq M_*/$M$_\odot<10^{'+str(log10_mass_min+1)+'}$')
            mask=np.log10(mass_stars)>=11.
            ydata=np.mean(gals['mass_stars_disc_sfh'][mask,:n_bin],0)/dt
        else:
            print('Plot type',plotName,'not yet implemented')
            continue # with next plot request
        plt.plot(xdata,ydata,plotProps['markerType'],label=plotProps['label'])
        if plotProps['legend']: plt.legend()
        if plotProps['save_png']: plt.savefig('figs/'+save_prefix+plotProps['name']+'.png',bbox_inches='tight')
        if plotProps['save_pdf']: plt.savefig('figs/'+save_prefix+plotProps['name']+'.pdf',bbox_inches='tight')